In [17]:
import sys
import numpy as np
np.set_printoptions(threshold=sys.maxsize)
import numpy.typing as npt
import cProfile

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 



In [6]:

class Biome():
    def __init__(self, grid: npt.NDArray[np.int8]):
        self.grid = grid
        self.val_grid = np.zeros(grid.shape, dtype=np.int8)

    def __repr__(self):
        return str(self.grid)
    
    def run(self, generations: int):
        for i in range(generations):
            self.grid = self._update(self.grid)
        
    def _update(self, grid: npt.NDArray[np.int8]) -> npt.NDArray[np.int8]:
        
        # get indices of all non-zero elements
        indices = np.argwhere(grid == 1)
   
        # get the surrounding indices
        surrounding_indices = np.array([[x + i, y + j] for x, y in indices for i in range(-1, 2) for j in range(-1, 2) if not (i == 0 and j == 0) ])
      
        # remove the indices that are out of bounds
        surrounding_indices = surrounding_indices[(surrounding_indices[:, 0] >= 0) 
                                                & (surrounding_indices[:, 0] < grid.shape[0])
                                                & (surrounding_indices[:, 1] >= 0) 
                                                & (surrounding_indices[:, 1] < grid.shape[1])]

        # get the unique surrounding indices
        each_indices = np.unique(surrounding_indices, axis=0)

        # set the surrounding indices to 1 if they have 3 neighbours
        grid[indices[:, 0], indices[:, 1]] = [1 if np.sum(np.all(surrounding_indices == i, axis=1)) == 2 else 0 for i in np.unique(indices, axis=0)]
        grid[each_indices[:, 0], each_indices[:, 1]] = [grid[i[0], i[1]] if np.sum(np.all(surrounding_indices == i, axis=1)) != 3 else 1 for i in each_indices  ]
       
        return grid
    
    def values(self, grid: npt.NDArray[np.int8]) -> npt.NDArray[np.int8]:
        # get indices of all non-zero elements
        indices = np.argwhere(grid == 1)
   
        # get the surrounding indices
        surrounding_indices = np.array([[x + i, y + j] for x, y in indices for i in range(-1, 2) for j in range(-1, 2) if not (i == 0 and j == 0) ])
      
        # remove the indices that are out of bounds
        surrounding_indices = surrounding_indices[(surrounding_indices[:, 0] >= 0) 
                                                & (surrounding_indices[:, 0] < grid.shape[0])
                                                & (surrounding_indices[:, 1] >= 0) 
                                                & (surrounding_indices[:, 1] < grid.shape[1])]
        
        val_grid = grid.copy()

        for i in np.unique(surrounding_indices, axis=0):
            val_grid[i[0], i[1]] = np.sum(np.all(surrounding_indices == i, axis=1))
        
        return val_grid



In [25]:
grid = [12,
        13,
        4,
        2]

def num_to_binary(num: int, length: int) -> npt.NDArray[np.int8]:
    return np.array([int(i) for i in np.binary_repr(num, length)])

def binary_to_num(binary: npt.NDArray[np.int8]) -> int:
    return int("".join([str(i) for i in binary]), 2)

def grid_to_binary(grid: list) -> npt.NDArray[np.int8]:
    return np.array([num_to_binary(i, 4) for i in grid])

def binary_to_grid(binary: npt.NDArray[np.int8]) -> list:
    return [binary_to_num(i) for i in binary]




print(grid_to_binary(grid))
num_to_binary((13 << 1), 4)


[[1 1 0 0]
 [1 1 0 1]
 [0 1 0 0]
 [0 0 1 0]]


array([1, 1, 0, 1, 0])

In [27]:
import bitarray as ba
